# NEXUS tool: case study for the Jordan - energy demand calculations
In this notebook a case study for the Jordan country is covered using the `nexustool` package. The water requirements for agricultural irrigation, residential, industrial and tourism use were previously calculated using the Water Evaluation and Planning System (WEAP) model. In this case study, the energy requirements for groundwater pumping, wastewater treatment, desalination of seawater and pumping for water conveyance are estimated.

First import the package by running the following block:

In [ ]:
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import os
import nexustool
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from scripts.functions import *

## 1. Read scenario data
After importing all required packages. Change the `data_folder` and `scenario` variables to reflect the name and relative location of your data file. This dataset should already have the water demand for irrigation estimations.

In [ ]:
data_folder = os.path.join('data', 'processed results')
scenario = 'Reference'
input_folder = os.path.join(data_folder, scenario)

## 2. Create nexus model 
To create a model simply create an instance of the `nexustool.Model()` class and store it in a variable name. The `nexustool.Model()` class requires a dataframe as input data. We wil create a model using the `pipelines_flow.gz` data, for this, we read the dataset into a `pandas` dataframe `df`:

In [ ]:
file_path = os.path.join(input_folder, 'pipelines_flow.gz')
df = pd.read_csv(file_path)
jordan = nexustool.Model(df)

## 3. Define variable names
The names of the properties of the model can be changed at any time. This is important for the model to know how each property is called withing your input data. To check the current property names run the `.print_properties()` method, a list with the names of each property and its current value will be displayed.

Then you can provide the right names for each property, calling them and assigning a value as:
```python
jordan.elevation = 'elevation_delta'
jordan.gw_depth = 'name_of_ground_water_depth'
```

In this particular case we will need to change the following default values:

In [ ]:
jordan.elevation_diff = 'elevation_delta' # for the case of GW, the elevation_diff is set to be the wtd
jordan.L = 'segment_length' # for the case of GW, the distance is set to be the wtd
jordan.D = 'Pipe_diameter'

# Defines the name of the variable for Peak Water Demand and Seasonal Scheme Water demand (monthly)
jordan.sswd = 'sswd' # Seassonal Scheme Water Demand
jordan.pa_e = 'pa_e' # Pumping Average Energy

## 4. Define pipelines diameters and pumping efficiency
Now we need to define the specifications of the water network, giving pipeline / canal diameter values and surface pumping efficiency:

In [ ]:
# General specifications fo all pipelines
jordan.set_specifications(category='pipeline', diameter=1.2, 
                          amount=1, pumping_hours_per_day=20, pump_efficiency=0.5)

# Specific specifications for the King Abdullah Canal (KAC)
jordan.set_specifications(category='pipeline', names='KAC', 
                          diameter=4, pumping_hours_per_day=20)

# Specific specifications for the Disi and Red - Dead pipelines
jordan.set_specifications(category='pipeline', names=['PL_Disi', 'PL_RedDead'], 
                          diameter=1.8, amount=1, pumping_hours_per_day=10, pump_efficiency=0.8)

# Specific specifiations for the Zara Ma'in pipeline
jordan.set_specifications(category='pipeline', names='PS_ZaraMain', 
                          diameter=1.2, amount=1, pumping_hours_per_day=24, pump_efficiency=0.8)

## 5. Setting a pumping efficiency goal for the water network

In [ ]:
efficiency_goal = {'efficiency': 0.5,
                   'init_year': 2020,
                   'end_year': 2050}
jordan.set_efficiency_goal('pipeline', efficiency_goal)

## 5. Calculate pumping energy requirements
To estimate the pumping energy requirements for conveyance, first we need to calculate the Total Dinamic Head (TDH). This is a measure in meters that accounts for the elevation difference between two points and the pressure loss in distribution.

For that, the area $A$ `.pipe_area()`, the velocity $V$ `.flow_velocity()`, the Reynolds number $Re$ `.reynolds()` and the friction factor $f$ `.friction_factor()` need to be estimated. The `nexustool` provides simple functions that allows us to make an easy estimation of these variables, which have the following formulas implemented in the background:

$$
A\,(m^2) = \pi\cdot \frac{D^2}{4}
$$

$$
V\,(m/s) = \frac{SSWD\,(m^3/month)}{PumpHours\,(h/day)\cdot 30\,(day/month)\cdot 3600\,(s/h)\cdot A\,(m^2)}
$$

$$
Re = \frac{V\,(m/s)\cdot D\,(m)}{v\,(m^2/s)}
$$

Where $v$ is the kinematic viscosity of water at around 1.004e-06 m<sup>2</sup>/s. And the friction factor is estimated according to the Swamee–Jain equation:

$$
f = \frac{0.25}{\left[log_{10}\left(\frac{\epsilon}{3.7D}+\frac{5.74}{Re^{0.9}}\right)\right]^2}
$$

Where $\epsilon$ is the roughness of the material (assumed at 0.061 mm for Ductile iron pipe). 

In [ ]:
jordan.pipe_area()
jordan.flow_velocity()
jordan.reynolds()
jordan.friction_factor()

Then, the TDH can be calculated by simply calling the `.get_tdh()` function.

$$
TDH\,(m) = \Delta H + f\cdot \frac{L\,(m)}{D\,(m)}\cdot \frac{V(m/s)^2}{2\cdot g\,(m/s^2)}
$$

Where $\Delta H$ is the elevation difference between start and end points of conveyance.

The conveyance pumping energy requirements are the n calculated by calling the `.get_pumping_energy()` method. The equation used to calculate the Electricity Demand ($E_D$) for pumping is as follows:

$$
E_D\,(kW_h) = \frac{SSWD\,(m^3)\cdot \rho\,(kg/m^3)\cdot g\,(m/s^2)\cdot TDH\,(m)}{PP_{eff}\,(\%)\cdot 3600\,(s/h)\cdot 1000\,(W/kW)}
$$

The variable withing the Model representing the $E_D$ is the `pa_e` or Pumping Average Electricity requirements.

Moreover, the Power Demand for pumping ($PD$) is denoted by the variable `pp_e` and calculated by the following formula:

$$
PD\,(kW) = \frac{PWD\,(m^3/s)\cdot \rho\,(kg/m^3)\cdot g\,(m/s^2)\cdot TDH\,(m)}{PP_{eff}\,(\%)\cdot 1000\,(W/kW)}
$$

The `.get_pumping_energy()` method calculates both the $E_D$ (`pa_e`) and $PD$ (`pp_e`).

In [ ]:
jordan.get_tdh(friction=True)
jordan.get_pumping_energy()

## 6. Create nexus model for groundwater pumping
The steps needed to estimate the groundwater pumping energy requirements are farily similar to the previous ones. We need to set the path to the `groundwater_supply.csv`, read it into a dataframe and create a nexus Model with it. Then, we define the pipe diameter and some required atttributes names:

In [ ]:
file_path = os.path.join(input_folder, 'groundwater_supply.gz')
df_groundwater = pd.read_csv(file_path)

jordan_gw = nexustool.Model(df_groundwater)

jordan_gw.elevation_diff = 'wtd_m'
jordan_gw.L = 'wtd_m'

# Defines the name of the variable for Peak Water Demand and Seasonal Scheme Water demand (monthly)
jordan_gw.sswd = 'sswd' # Seassonal Scheme Water Demand
jordan_gw.pa_e = 'pa_e' # Pumping Average Energy

# Set general specifications for groundwater wells. We pass the total amount of wells in the country 
# and distribute the mproportionally based on water extracion levels of each groundwater basin
jordan_gw.set_specifications(category='well', diameter=0.1, 
                             amount=5000, distribute='proportionally', 
                             pumping_hours_per_day=5, pump_efficiency=0.45)

# Set specifications for the wells of the Disi aquifer 
jordan_gw.set_specifications(category='well', diameter=0.3, pumping_hours_per_day=5, 
                             pump_efficiency=0.8, amount=55, names='GW_Southern Desert_Ram')

## 8. Setting a pumping efficiency goal for wells

In [ ]:
efficiency_goal = {'efficiency': 0.45,
                   'init_year': 2020,
                   'end_year': 2050}
jordan_gw.set_efficiency_goal('well', efficiency_goal)

## 7. Calculate groundwater pumping energy
The energy requirements are then calculated calling the `.pipe_area()`, `.flow_velocity()`, `.reynolds()`, `.friction_factor()`, `.get_tdh()` and get `.get_pumping_energy()` methods.

In [ ]:
jordan_gw.pipe_area()
jordan_gw.flow_velocity()
jordan_gw.reynolds()
jordan_gw.friction_factor()

jordan_gw.get_tdh(friction=True)
jordan_gw.get_pumping_energy()

## 8. Calculate wastewater treatment energy
To estimate the energy requirements for wastewater treatment, we read the `wwtp_inflow.csv` data into a dataframe and create a nexus Model. Set the specifications of the wastewater treatment plants using the `.set_treatment_energy` method. This method accepts a category (wastewater or desalination), an energy intensity of treatment (defined in kWh/m<sup>3</sup>) and a name or list of names (`names`) for the plants you whish to set specifications for. 

The `.get_treatment_energy` method, calculates the total energy requirements multiplying the `sswd` attribute (Seassonal Water Demand), wich represents the amount of treated water, by each defined energy intensity.

In [ ]:
file_path = os.path.join(input_folder, 'wwtp_inflow.gz')
df_wwtp = pd.read_csv(file_path)

jordan_ww = nexustool.Model(df_wwtp)
jordan_ww.sswd = 'sswd' # Seassonal Scheme Water Demand
jordan_ww.pa_e = 'pa_e' # Pumping Average Energy

# Add info on treatment energy needs. This is done depending on the treatment technology used by the plant
# In general it is assumed that all plants use the activate sludge treatment with an average energy needs of 0.8 kwh/m3
jordan_ww.set_treatment_energy(category='wastewater', energy_int=0.8)

# For the Samra wastewater treatment plant, we use a lower energy intensity at 0.6 kwh/m3
# as that plant uses to the energy recovery
jordan_ww.set_treatment_energy(category='wastewater', energy_int=0.6, names='WWTP_Samra')

# For the Al Baqa wastewater treatment plant, the trickling filter treatment is assumed 
# with an energy intensity of 0.4 kwh/m3
jordan_ww.set_treatment_energy(category='wastewater', energy_int=0.4, names='WWTP_Al Baqa')

# Calculates the energy needs for treatment
jordan_ww.get_treatment_energy(category='wastewater')

## 9. Calculate desalination energy
Finally, we need to calculate the energy requirements for desalination making use of the `desalination.gz` dataset to create a nexus Model and following the same steps described for the wastewater treatment plants.

In [ ]:
file_path = os.path.join(input_folder, 'desalination.gz')
df_desal = pd.read_csv(file_path)

jordan_desal = nexustool.Model(df_desal)
jordan_desal.sswd = 'sswd' # Seassonal Scheme Water Demand
jordan_desal.pa_e = 'pa_e' # Pumping Average Energy

# For the Red-Dead desalination project we assume an energy intensity of 3.31 kwh/m3 based on avaiable information
jordan_desal.set_treatment_energy(category='desalination', energy_int=3.31, names='RedDead')

# For the Aqaba desalination plant we assume an energy intensity of 5 kwh/m3 
# based on the high salinity content of the treated water
jordan_desal.set_treatment_energy(category='desalination', energy_int=5, names='Aqaba Desal')

# Calculates the energy needs for treatment
jordan_desal.get_treatment_energy(category='desalination')

## 10. Save result files
The results can then be saved into a defined output folder `results_folder`. The `save_results` function can be used to make this process easy. It takes a path for a `results_folder` and saves all of the result files there.

In [ ]:
subscenario = 'Current Efficiency'
results_folder = os.path.join('results', f'{scenario} - {subscenario}')

save_results(jordan, jordan_gw, jordan_ww, jordan_desal, results_folder, input_folder)

## 11. Plot results
To display results for the scenario just run the following cells:

### Water deliveries

In [ ]:
water_delivered = pd.read_csv(os.path.join(results_folder, 'water_delivered.gz'))
dff_delivered = water_delivered.groupby(['Year', 'type'])['sswd'].sum() / 1000000
dff_delivered = dff_delivered.reset_index()

fig = px.bar(dff_delivered, x='Year', y='sswd', color='type',
             color_discrete_sequence=px.colors.qualitative.Dark2)
fig.update_layout(title=f'Water delivered {scenario} scenario', yaxis_title='Water (Mm<sup>3</sup>)')

### Energy demand

In [ ]:
dff_energy = pd.DataFrame()
jordan.df['type'] = 'Water conveyance'

for df in [jordan_ww.df, jordan_desal.df, jordan_gw.df, jordan.df]:
    dff = df.groupby(['Year', 'type'])['pa_e'].sum() / (1000000)
    dff = dff.reset_index()
    dff_energy = dff_energy.append(dff, sort=False)

fig = px.bar(dff_energy, x='Year', y='pa_e', color='type',
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(title=f'Energy demand {scenario} scenario', yaxis_title='Energy (GWh)')

### Crop production

In [ ]:
crop_production = pd.read_csv(os.path.join(results_folder, 'crop_production.gz'))
df_production = crop_production.groupby(['Year', 'variable']).sum() / 1000000
df_production.reset_index(inplace=True)
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
          '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff',
          '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1',
          '#000075', '#808080', '#ffffff', '#000000']
fig = px.bar(df_production, x='Year', y='production',
             color='variable',
             color_discrete_sequence=colors)
fig.update_layout(title=f'Crop production {scenario} scenario', yaxis_title='Production (kton)')

### Compare results
You can compare general results for several scenarios. These include unmet demands for agriculture and municipalities, overall energy demand, and agricultural water productivity (crop per drop).
#### Read the scenarios data
First you will need to read the data of previously run scenarios. In the `directory_folder` variable, specify the folder containing the data folder with the scenarios results.

Then add to the `scenarios` variable, the name of each scenario that you want to compare. for example:
```python
scenarios = ['Reference',
             'New Resources',
             'Increased Water Productivity',
             'Reduce NRW to 20 percent']
```

In [ ]:
directory_folder = 'results'
scenarios = ['Reference - Current Efficiency',
             'Reference - 80% Efficiency']

df_delivered, df_required, df_gw, df_pipelines, df_wwtp, df_desal, df_crop = merge_scenario_data(directory_folder, scenarios)

names = {'sswd': 'Water delivered (Mm<sup>3</sup>)', 'production_kg': 'Crop production (ton)', 
         'swpa_e': 'Energy demand (GWh)', 'unmet_demand_year': 'Annual unmet demand (%)',
         'wtd_m': 'Depth to groundwater (meters)'}

category_orders = {'Climate': ['Historical Trend', 'Climate Change'],
                   'Scenario': ['Reference', 'Desalination'],
                   'type': ['Groundwater pumping', 'Surface water conveyance', 'Wastewater treatment',
                            'Desalination', 'Desalinated water conveyance']}

#### Comparing unmet demand
Run the folowing cell:

In [ ]:
dff_delivered = df_delivered.groupby(['Scenario', 'Year', 'type'])['sswd'].sum() / 1000000
dff_delivered = dff_delivered.reset_index()
dff_required = df_required.groupby(['Scenario', 'Year', 'type'])['sswd'].sum() / 1000000
dff_required = dff_required.reset_index()

dff_unmet = dff_required.copy()
dff_unmet['value'] = (dff_unmet['sswd'] - dff_unmet.set_index(['Scenario', 'Year', 'type']).index.map(
    dff_delivered.set_index(['Scenario', 'Year', 'type'])['sswd'])) / dff_unmet['sswd']

sectors = ['Agriculture', 'Municipality']
dff = dff_unmet.reset_index()
dff = dff.loc[dff['type'].isin(sectors)]

fig = px.line(dff, x='Year', y='value', color='Scenario', facet_row='type',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_yaxes(matches=None)
fig.update_traces(line=dict(width=3))
for axis in fig.layout:
    if (type(fig.layout[axis]) == go.layout.YAxis):
        fig.layout[axis].title.text = 'Unmet demand (%)'
        fig.layout[axis].tickformat = '%'
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

#### Comparing energy demand
Run the foloing cell:

In [ ]:
dff_energy = pd.DataFrame()

for df in [df_wwtp, df_desal, df_gw, df_pipelines]:
    dff = df.groupby(['Scenario', 'Year'])['pa_e'].sum() / (1000000)
    dff = dff.reset_index()
    dff_energy = dff_energy.append(dff, sort=False)

dff = dff_energy.groupby(['Scenario', 'Year']).agg({'pa_e': 'sum'}).reset_index()

fig = px.line(dff, x='Year', y='pa_e', color='Scenario', 
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)

fig.update_layout(yaxis_title_text='Energy demand (GWh)', showlegend=True)
fig.update_traces(line=dict(width=3))

#### Comparing agricultural water productivity
Run the foloing cell:

In [ ]:
dff_delivered = df_delivered.loc[df_delivered['type']=='Agriculture']
dff_delivered = dff_delivered.groupby(['Scenario', 'Year']).agg({'sswd': lambda r: sum(r) / 1000000}).reset_index()
dff_crop = df_crop.groupby(['Scenario', 'Year']).agg({'production': lambda r: sum(r) / 1000000}).reset_index()
dff = dff_delivered[['Scenario', 'Year']]
dff['value'] = dff_crop['production'] / dff_delivered['sswd']

fig = px.line(dff, x='Year', y='value', color='Scenario',
              color_discrete_sequence=px.colors.qualitative.T10, 
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(yaxis_title_text='Crop per drop (kg/m<sup>3</sup>)')
fig.update_traces(line=dict(width=3))